# Dart 전자공시를 활용하여 자료를 가져오는 방법

In [27]:
import requests
import pandas as pd
import zipfile
import xml.etree.ElementTree as ET
import os

In [37]:
api_key = '0728a6d81b70b86763fd38daafcc1fccff0beedb'

In [39]:
# 기업 고유번호 가져오는 방법.
# 개발가이드 https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018

# 기업 고유번호 XML 파일 다운로드 URL
url = f'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}'

# XML 파일 다운로드
response = requests.get(url)
zip_path = 'corpCode.zip'

with open(zip_path, 'wb') as file:
    file.write(response.content)

# ZIP 파일 압축 해제
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

# XML 파일 파싱
xml_file = 'CORPCODE.xml'
tree = ET.parse(xml_file)
root = tree.getroot()

# 삼성전자의 고유번호 찾기
corp_name = '삼성전자'
corp_code = None

for company in root.findall('list'):
    if company.find('corp_name').text == corp_name:
        corp_code = company.find('corp_code').text
        break

if corp_code:
    print(f"삼성전자의 고유번호: {corp_code}")
else:
    print("삼성전자를 찾을 수 없습니다.")

# 사용이 끝난 파일 삭제
os.remove(zip_path)
os.remove(xml_file)


삼성전자의 고유번호: 00126380


In [49]:
# 기업개황 API 사용법
# 개발가이드 https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019002#none

# DART API endpoint for retrieving disclosures
url = 'https://opendart.fss.or.kr/api/company.json'

# Define parameters
params = {
    'crtfc_key': api_key,       # Your DART API key
    'corp_code': corp_code,     # Corporate code for Samsung Electronics
}

# Send request
response = requests.get(url, params=params)
data = response.json()

# Check if response contains disclosures
if data.get('status') == '000':  # Status '000' indicates success
    # Convert to DataFrame for better display
    disclosures = pd.DataFrame([data])
    print(disclosures)
else:
    print("Error:", data.get('message'))


  status message corp_code corp_name                corp_name_eng stock_name  \
0    000      정상  00126380   삼성전자(주)  SAMSUNG ELECTRONICS CO,.LTD       삼성전자   

  stock_code ceo_nm corp_cls       jurir_no     bizr_no  \
0     005930    한종희        Y  1301110006246  1248100998   

                        adres               hm_url ir_url        phn_no  \
0  경기도 수원시 영통구  삼성로 129 (매탄동)  www.samsung.com/sec         02-2255-0114   

         fax_no induty_code    est_dt acc_mt  
0  031-200-7538         264  19690113     12  


In [53]:
disclosures.head(1)

,status,message,corp_code,corp_name,corp_name_eng,stock_name,stock_code,ceo_nm,corp_cls,jurir_no,bizr_no,adres,hm_url,ir_url,phn_no,fax_no,induty_code,est_dt,acc_mt
0,000,정상,00126380,삼성전자(주),"SAMSUNG ELECTRONICS CO,.LTD",삼성전자,005930,한종희,Y,1301110006246,1248100998,경기도 수원시 영통구 삼성로 129 (매탄동),www.samsung.com/sec,,02-2255-0114,031-200-7538,264,19690113,12


In [79]:
# DART API 엔드포인트 (재무정보 지표)
# 개발가이드 https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2019016

url = 'https://opendart.fss.or.kr/api/fnlttSinglAcnt.json'

# 요청 파라미터 설정
params = {
    'crtfc_key': api_key,
    'corp_code': corp_code,
    'bsns_year': '2024',
    'reprt_code': '11013',  # 1분기 보고서 코드
    'fs_div': 'CFS'  # 연결재무제표 (CFS)
}

# API 요청
response = requests.get(url, params=params)
data = response.json()

# 응답 상태 확인
if data.get('status') == '000':  # 성공 상태 코드
    balancesheet = pd.DataFrame(data['list'])
else:
    print("API 오류:", data.get('message'))


In [81]:
balancesheet.head()

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,ord,currency,thstrm_add_amount,frmtrm_add_amount
0,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,제 56 기1분기말,2024.03.31 현재,"208,544,280,000,000",제 55 기말,2023.12.31 현재,"195,936,557,000,000",1,KRW,NaN,NaN
1,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,제 56 기1분기말,2024.03.31 현재,"262,355,532,000,000",제 55 기말,2023.12.31 현재,"259,969,423,000,000",3,KRW,NaN,NaN
2,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,제 56 기1분기말,2024.03.31 현재,"470,899,812,000,000",제 55 기말,2023.12.31 현재,"455,905,980,000,000",5,KRW,NaN,NaN
3,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,제 56 기1분기말,2024.03.31 현재,"81,770,355,000,000",제 55 기말,2023.12.31 현재,"75,719,452,000,000",7,KRW,NaN,NaN
4,20240516001421,11013,2024,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,제 56 기1분기말,2024.03.31 현재,"17,213,333,000,000",제 55 기말,2023.12.31 현재,"16,508,663,000,000",9,KRW,NaN,NaN


In [95]:
# 지표 사용 관련 개발가이드 https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS003&apiId=2022001

# 보고서코드 : 1분기  // 11013
# 보고서코드 : 반기   // 11012
# 보고서코드 : 3분기  // 11014
# 보고서코드 : 사업보고서 // 11011

# 수익성지표 : M210000 안정성지표 : M220000 성장성지표 : M230000 활동성지표 : M240000


url = 'https://opendart.fss.or.kr/api/fnlttSinglIndx.json'

# 요청 파라미터 설정
params = {
    'crtfc_key': api_key,
    'corp_code': corp_code,
    'bsns_year': '2024',
    'reprt_code': '11013',  # 1분기 보고서 코드
    'idx_cl_code': 'M210000'  # 수익성지표 (CFS)
}

# API 요청
response = requests.get(url, params=params)
data = response.json()

# 응답 상태 확인
if data.get('status') == '000':  # 성공 상태 코드
    index_class = pd.DataFrame(data['list'])
else:
    print("API 오류:", data.get('message'))


In [99]:
index_class

,reprt_code,bsns_year,corp_code,stock_code,stlm_dt,idx_cl_code,idx_cl_nm,idx_code,idx_nm,idx_val
0,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211100,세전계속사업이익률,9.393
1,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211200,순이익률,9.393
2,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211250,총포괄이익률,16.338
3,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211300,매출총이익률,36.194
4,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211400,매출원가율,63.806
5,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211550,ROE,1.837
6,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M211800,판관비율,27.008
7,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M212000,총자산영업이익률,1.426
8,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M212100,총자산세전계속사업이익률,1.458
9,11013,2024,00126380,005930,2024-03-31,M210000,수익성지표,M212200,자기자본영업이익률,1.796
